# Results

In [344]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


from tensorflow.keras import models
from tensorflow.keras.layers import LSTM,Dense

In [345]:
df_securities = pd.read_csv('./data/securities.csv')
df_prices = pd.read_csv('./data/prices-split-adjusted.csv')
df_prices.fillna(float(0))

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [346]:
def get_companies(num):
    choices = []
    names = []
    while(len(choices) < 5):
        i = random.randint(0,df_securities.shape[0]-1)
        symbol = df_securities['Ticker symbol'].iloc[i]
        name = df_securities['Security'].iloc[i]
        temp_df = df_prices[df_prices['symbol']==symbol]
        if (temp_df.shape[0] > 1500) and (symbol not in choices):
            choices.append(symbol)
            names.append(name)
        
    return choices, names


def get_multiple_df(symbols):
    directory = []
    
    for symbol in symbols:
        single_df = df_prices[df_prices['symbol']==symbol]
        single_df = single_df.drop(columns=['symbol'])
        single_df = single_df.set_index('date')
        single_df = single_df['close'].to_frame()
        
        directory.append(single_df)
    return directory


def fix_data(data, scalar):
    return scalar.inverse_transform(np.array(data).reshape(-1,1))


def build_model(optimizer, step):
    model = models.Sequential()

    model.add(LSTM(64,return_sequences=True,input_shape=(step,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(32,return_sequences=True))
    model.add(LSTM(50,return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    
    return model


def prep_forecast(prediction_list, scalar, split):
    step = 10
    
    prediction_list = scalar.fit_transform(np.array(prediction_list).reshape(-1,1))
    size = int(len(prediction_list)*split)
    
    forecast_data = prediction_list[0:size,:] #data
    
    x = []
    for i in range(len(prediction_list)-step-1):
        k = i+step
        x.append(forecast_data[i:k,0])
    x = np.array(x)
    forecast_data = np.reshape(x,(x.shape[0],x.shape[1],1))
    
    return forecast_data


class Company:
    def __init__(self, symbol, name, df):
        self.symbol = symbol
        self.name = name
        self.df = df
        self.model = None
        
        self.train_data = []
        self.test_data = []
        
        self.x = []
        self.y = []
        
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
        self.y_hat = None
        self.error = None
        
        self.history = None
        self.forcast = None
        
        
    def prep_data(self, scalar, split):
        scaled_df = scalar.fit_transform(np.array(self.df).reshape(-1,1))
        size = int(len(self.df)*split)

        self.train_data = scaled_df[0:size,:]
        self.test_data = scaled_df[size:,:]
        
    
    def build_data(self, step, history_div):
        x, y = [], []
        for i in range(int(len(self.train_data)-step-1)):
            k = i+step
            x.append(self.train_data[i:k,0])
            y.append(self.train_data[k,0])
            
        X_train = np.array(x)
        X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))  
        self.X_train = X_train
        self.y_train = np.array(y)

        x, y = [], []
        for i in range(int(len(self.test_data)-step-1)):
            k = i+step
            x.append(self.test_data[i:k,0])
            y.append(self.test_data[k,0])
            
        X_test = np.array(x)
        X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)      
        self.X_test = X_test
        self.y_test = np.array(y)
        self.history = int(len(self.y_test)/history_div)
        
        
    def train_model(self, step, scalar, epochs, batch):
        self.model = build_model('adam',step)
        self.model.fit(self.X_train,self.y_train,epochs=epochs,batch_size=batch)

        self.y_hat = self.model.predict(self.X_test)
        self.error = np.sqrt(mean_squared_error(self.y_test,self.y_hat))

        self.y_test = fix_data(self.y_test, scalar)
        self.y_hat = fix_data(self.y_hat, scalar)
        print(self.error)
        
        
    def get_forecast(self, scalar):
        prediction_list = self.y_test[-self.history:]
        forecast_data = prep_forecast(prediction_list, scalar, 1)
        
        forecast_data = self.model.predict(forecast_data)
        forecast_data = fix_data(forecast_data, scalar)
        self.forecast = forecast_data

In [347]:
directory_size = 5
directory = []

split = 0.7
step = 60
epochs = 60
batch = 10
history_div = 23

scalar = sklearn.preprocessing.MinMaxScaler()

symbols, names = get_companies(directory_size)
selected_dfs = get_multiple_df(symbols)
for i in range(directory_size):
    directory.append(Company(symbols[i], names[i], selected_dfs[i]))
    
for company in directory:
    print("============================================================")
    
    company.prep_data(scalar, split)
    company.build_data(step, history_div)
    
    print(company.symbol+": "+company.name)
    print("X Training Shape: "+str(company.X_train.shape))
    print("X Test Shape: "+str(company.X_test.shape))
    print("Y Training Shape: "+str(company.y_train.shape))
    print("Y Test Shape: "+str(company.y_test.shape))
    
    company.train_model(step, scalar, epochs, batch)

EW: Edwards Lifesciences
X Training Shape: (1172, 60, 1)
X Test Shape: (468, 60, 1)
Y Training Shape: (1172,)
Y Test Shape: (468,)
Epoch 1/60
118/118 [==============================] - 13s 113ms/step - loss: 0.0024
Epoch 2/60
118/118 [==============================] - 14s 117ms/step - loss: 7.2219e-04
Epoch 3/60
118/118 [==============================] - 14s 120ms/step - loss: 6.7245e-04
Epoch 4/60
118/118 [==============================] - 15s 124ms/step - loss: 6.8762e-04
Epoch 5/60
118/118 [==============================] - 13s 114ms/step - loss: 4.7183e-04
Epoch 6/60
118/118 [==============================] - 15s 125ms/step - loss: 4.3673e-04
Epoch 7/60
118/118 [==============================] - 13s 110ms/step - loss: 3.5460e-04
Epoch 8/60
118/118 [==============================] - 13s 106ms/step - loss: 2.8961e-04
Epoch 9/60
118/118 [==============================] - 13s 113ms/step - loss: 2.5227e-04
Epoch 10/60
118/118 [==============================] - 13s 112ms/step - loss: 2.6

118/118 [==============================] - 12s 103ms/step - loss: 1.3159e-04
Epoch 29/60
118/118 [==============================] - 12s 104ms/step - loss: 1.3155e-04
Epoch 30/60
118/118 [==============================] - 12s 103ms/step - loss: 1.4509e-04
Epoch 31/60
118/118 [==============================] - 12s 104ms/step - loss: 1.3664e-04
Epoch 32/60
118/118 [==============================] - 12s 103ms/step - loss: 1.2891e-04
Epoch 33/60
118/118 [==============================] - 12s 104ms/step - loss: 1.5707e-04
Epoch 34/60
118/118 [==============================] - 12s 104ms/step - loss: 1.5070e-04
Epoch 35/60
118/118 [==============================] - 12s 104ms/step - loss: 1.3656e-04
Epoch 36/60
118/118 [==============================] - 12s 103ms/step - loss: 1.5287e-04
Epoch 37/60
118/118 [==============================] - 12s 103ms/step - loss: 1.4145e-04
Epoch 38/60
118/118 [==============================] - 12s 103ms/step - loss: 1.3802e-04
Epoch 39/60
118/118 [============

118/118 [==============================] - 12s 100ms/step - loss: 2.4652e-04
Epoch 58/60
118/118 [==============================] - 12s 100ms/step - loss: 2.5110e-044s -
Epoch 59/60
118/118 [==============================] - 12s 99ms/step - loss: 2.2277e-04
Epoch 60/60
118/118 [==============================] - 12s 99ms/step - loss: 2.7426e-04
0.03976502522475036
HOLX: Hologic
X Training Shape: (1172, 60, 1)
X Test Shape: (468, 60, 1)
Y Training Shape: (1172,)
Y Test Shape: (468,)
Epoch 1/60
118/118 [==============================] - 12s 102ms/step - loss: 0.0033
Epoch 2/60
118/118 [==============================] - 12s 102ms/step - loss: 0.0015
Epoch 3/60
118/118 [==============================] - 12s 102ms/step - loss: 0.0011
Epoch 4/60
118/118 [==============================] - 12s 102ms/step - loss: 8.6710e-04
Epoch 5/60
118/118 [==============================] - 12s 102ms/step - loss: 7.3790e-04
Epoch 6/60
118/118 [==============================] - 12s 102ms/step - loss: 5.7402e-0

118/118 [==============================] - 12s 105ms/step - loss: 3.3737e-04
Epoch 25/60
118/118 [==============================] - 12s 105ms/step - loss: 1.2789e-04
Epoch 26/60
118/118 [==============================] - 12s 106ms/step - loss: 1.4293e-04
Epoch 27/60
118/118 [==============================] - 12s 106ms/step - loss: 1.3599e-04
Epoch 28/60
118/118 [==============================] - 12s 105ms/step - loss: 1.2669e-041s - los
Epoch 29/60
118/118 [==============================] - 12s 105ms/step - loss: 1.2252e-04
Epoch 30/60
118/118 [==============================] - 12s 105ms/step - loss: 1.3760e-04
Epoch 31/60
118/118 [==============================] - 12s 105ms/step - loss: 1.9435e-04
Epoch 32/60
118/118 [==============================] - 12s 106ms/step - loss: 1.0953e-04
Epoch 33/60
118/118 [==============================] - 12s 106ms/step - loss: 1.3888e-04
Epoch 34/60
118/118 [==============================] - 12s 105ms/step - loss: 1.3197e-04
Epoch 35/60
118/118 [====

In [ ]:
    company.train_model(step, scalar, epochs, batch)

In [ ]:
fig, ax = plt.subplots(directory_size,1, figsize = (20,directory_size*7.5))

for i in range(directory_size):
    if directory_size > 1:
        company = directory[i]

        ax[i].plot(company.y_test,label='Actual Price')
        ax[i].plot(company.y_hat, c='r', label='Predicted Price')
        ax[i].set_title(company.symbol+": "+company.name+' Stock Prediction')
        ax[i].set_ylabel('Stock Price')
        ax[i].set_xlabel('Time')
        ax[i].tick_params(axis='x',
                       which='both',
                       bottom=False,
                       top=False,
                       labelbottom=False)
        ax[i].legend()
    else:
        company = directory[i]

        ax.plot(company.y_test,label='Actual Price')
        ax.plot(company.y_hat, c='r', label='Predicted Price')
        ax.set_title(company.symbol+": "+company.name+' Stock Prediction')
        ax.set_ylabel('Stock Price')
        ax.set_xlabel('Time')
        ax.tick_params(axis='x',
                       which='both',
                       bottom=False,
                       top=False,
                       labelbottom=False)
        ax.legend()

In [ ]:
fig, ax = plt.subplots(directory_size,1, figsize = (20,directory_size*7.5))
for i in range(directory_size):
    company = directory[i]
    company.get_forecast(scalar)
    
    full_forecast = np.concatenate((company.y_test,company.forecast))
    
    graph_df = pd.DataFrame(full_forecast)
    
    if directory_size > 1:
        ax[i].plot(graph_df, label='Recorded Price History')
        ax[i].plot(graph_df.loc[graph_df.index >= len(company.y_test)], c='r',label='Future Prediction Price')

        ax[i].set_title('ExonMobil Corporation Stock Prediction')
        ax[i].set_ylabel('Stock Price')
        ax[i].set_xlabel('Time (2010-2016)')
        ax[i].tick_params(axis='x',
                       which='both',
                       bottom=False,
                       top=False,
                       labelbottom=False)
        ax[i].legend()
    else:
        ax.plot(graph_df, label='Recorded Price History')
        ax.plot(graph_df.loc[graph_df.index >= len(company.y_test)], c='r',label='Future Prediction Price')

        ax.set_title('ExonMobil Corporation Stock Prediction')
        ax.set_ylabel('Stock Price')
        ax.set_xlabel('Time (2010-2016)')
        ax.tick_params(axis='x',
                       which='both',
                       bottom=False,
                       top=False,
                       labelbottom=False)
        ax.legend()